<a href="https://colab.research.google.com/github/flahbocchino/farmtech-fase7-grupoFlavia-/blob/main/fase4_dashboard/FarmTech_Fase4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [2]:
df = pd.read_csv("base_fase1_com_meteo.csv")
df


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha,fertilizante_kg,temp_media_7d,chuva_total_7d,recomendacao_irrigacao
0,1,Soja,100,50,5000,0.5,25.0,22.423214,40.9,Irrigação reduzida
1,2,Milho,150,80,12000,1.2,84.0,22.423214,40.9,Irrigação reduzida
2,3,Cana,200,120,24000,2.4,216.0,22.423214,40.9,Irrigação reduzida


In [3]:
# Criar uma produtividade "de mentira", só pra fins de ML
# fórmula simples: quanto mais fertilizante e chuva moderada, maior a produtividade
df["produtividade_sacas_ha"] = (
    40
    + df["fertilizante_kg"] * 0.2
    - (df["temp_media_7d"] - 22) * 1.5
    + (df["chuva_total_7d"] * 0.3)
)

df


,talhao_id,cultura,comprimento_m,largura_m,area_m2,area_ha,fertilizante_kg,temp_media_7d,chuva_total_7d,recomendacao_irrigacao,produtividade_sacas_ha
0,1,Soja,100,50,5000,0.5,25.0,22.423214,40.9,Irrigação reduzida,56.635179
1,2,Milho,150,80,12000,1.2,84.0,22.423214,40.9,Irrigação reduzida,68.435179
2,3,Cana,200,120,24000,2.4,216.0,22.423214,40.9,Irrigação reduzida,94.835179


In [4]:
# Features (entradas) e alvo (produtividade)
X = df[["area_ha", "fertilizante_kg", "temp_media_7d", "chuva_total_7d"]]
y = df["produtividade_sacas_ha"]

# Como temos poucos dados, o split é mais simbólico, mas ok:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

modelo = RandomForestRegressor(random_state=42)
modelo.fit(X_train, y_train)

score = modelo.score(X_test, y_test)
score


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


nan

In [5]:
def prever_produtividade(area_ha, fertilizante_kg, temp_media, chuva_total):
    entrada = pd.DataFrame([{
        "area_ha": area_ha,
        "fertilizante_kg": fertilizante_kg,
        "temp_media_7d": temp_media,
        "chuva_total_7d": chuva_total
    }])
    pred = modelo.predict(entrada)[0]
    return round(pred, 2)

# Teste rápido
prever_produtividade(
    area_ha=1.0,
    fertilizante_kg=60,
    temp_media=22,
    chuva_total=40
)


np.float64(74.24)

In [6]:
dashboard_code = """
import streamlit as st
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Carrega dados
df = pd.read_csv("fase1_base_dados/base_fase1_com_meteo.csv")

# Cria variável de produtividade (mesma fórmula do notebook)
df["produtividade_sacas_ha"] = (
    40
    + df["fertilizante_kg"] * 0.2
    - (df["temp_media_7d"] - 22) * 1.5
    + (df["chuva_total_7d"] * 0.3)
)

X = df[["area_ha", "fertilizante_kg", "temp_media_7d", "chuva_total_7d"]]
y = df["produtividade_sacas_ha"]

modelo = RandomForestRegressor(random_state=42)
modelo.fit(X, y)

def prever_produtividade(area_ha, fertilizante_kg, temp_media, chuva_total):
    entrada = pd.DataFrame([{
        "area_ha": area_ha,
        "fertilizante_kg": fertilizante_kg,
        "temp_media_7d": temp_media,
        "chuva_total_7d": chuva_total
    }])
    pred = modelo.predict(entrada)[0]
    return round(pred, 2)

def main():
    st.title("Dashboard FarmTech - Fase 4")
    st.write("Visualização dos talhões e previsão de produtividade.")

    st.subheader("Dados atuais da fazenda")
    st.dataframe(df)

    st.subheader("Simular produtividade esperada")

    area_ha = st.number_input("Área (ha)", min_value=0.1, value=float(df["area_ha"].iloc[0]))
    fertilizante_kg = st.number_input("Fertilizante (kg)", min_value=0.0, value=float(df["fertilizante_kg"].iloc[0]))
    temp_media = st.number_input("Temperatura média 7 dias (°C)", value=float(df["temp_media_7d"].iloc[0]))
    chuva_total = st.number_input("Chuva total 7 dias (mm)", value=float(df["chuva_total_7d"].iloc[0]))

    if st.button("Prever produtividade"):
        prod = prever_produtividade(area_ha, fertilizante_kg, temp_media, chuva_total)
        st.success(f"Produtividade estimada: {prod} sacas/ha")

if __name__ == "__main__":
    main()
"""

dashboard_code[:500]


'\nimport streamlit as st\nimport pandas as pd\nfrom sklearn.ensemble import RandomForestRegressor\n\n# Carrega dados\ndf = pd.read_csv("fase1_base_dados/base_fase1_com_meteo.csv")\n\n# Cria variável de produtividade (mesma fórmula do notebook)\ndf["produtividade_sacas_ha"] = (\n    40\n    + df["fertilizante_kg"] * 0.2\n    - (df["temp_media_7d"] - 22) * 1.5\n    + (df["chuva_total_7d"] * 0.3)\n)\n\nX = df[["area_ha", "fertilizante_kg", "temp_media_7d", "chuva_total_7d"]]\ny = df["produtividade_sacas_ha"]\n\nmodel'